In [1]:
import gymnasium as gym

env = gym.make('CartPole-v1')

In [2]:
episodes = 5
cumulative_reward = 0

for e in range(episodes):
    state = env.reset()
    done = False
    episode_reward = 0

    while not done:
        action = env.action_space.sample()
        result = env.step(action)
        state = result[0]
        done = result[2]
        episode_reward += result[1]

    cumulative_reward += episode_reward
    print(f'Episode {e + 1} reward: {episode_reward}')

print(f'Average reward: {cumulative_reward / episodes}')
env.close()

Episode 1 reward: 23.0
Episode 2 reward: 13.0
Episode 3 reward: 17.0
Episode 4 reward: 18.0
Episode 5 reward: 16.0
Average reward: 17.4


In [11]:
import tensorflow as tf
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import RMSprop

from keras import __version__
tf.keras.__version__ = __version__
from rl.agents.dqn import DQNAgent
from rl.memory import SequentialMemory
from rl.policy import EpsGreedyQPolicy, LinearAnnealedPolicy

